# 自动求梯度

## 创建一个 Tensor 并设置 requires_grad=True
### Function 是另外一个很重要的类。 Tensor 和 Function 互相结合就可以构建一个记录有整个计算过  程的有向无环图(DAG)。每个 Tensor 都有一个 .grad_fn 属性,该属性即创建该 Tensor 的  Function , 就是说该 Tensor 是不是通过某些运算得到的,若是,则 grad_fn 返回一个与这些运算相关的对象,否则是None。

In [16]:
import torch
import numpy as np

In [17]:
x = torch.ones(2, 2, requires_grad=True)
print(x)
print(x.grad_fn)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
None


In [18]:
y = x + 2
print(y)
print(y.grad_fn) # AddBackward0

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


In [19]:
print(x.is_leaf, y.is_leaf)

True False


In [20]:
z = y * y * 3
out = z.mean()
print(z, out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward0>)


## 通过 .requires_grad_() 来用in-place的方式改变 requires_grad 属性:

In [30]:
a = torch.randn(2, 2) # 缺失的情况下默认False
a = ((a * 3) / (a - 1))
print(a.requires_grad)
b = (a * a).sum()
print(b.grad_fn) # 默认关闭时，无法追踪到
a.requires_grad_(True)
print(a.requires_grad)
b = (a * a).sum()
print(b.grad_fn)

False
None
True


## 梯度

In [ ]:
out.backward()
print(x.grad)

In [28]:
out2 = x.sum()
out2.backward() 
print(x.grad) # 这里的梯度和之前的out.backward()累加了

out3 = x.sum()
x.grad.data.zero_() # 梯度清零
out3.backward() 
print(x.grad)

tensor([[2., 2.],
        [2., 2.]])
tensor([[1., 1.],
        [1., 1.]])


In [33]:
x = torch.tensor([1.0, 2, 3, 4], requires_grad=True)
y = 2 * x
z = y.view(2, 2)
print(z)

tensor([[2., 4.],
        [6., 8.]], grad_fn=<ViewBackward0>)


In [34]:
v = torch.tensor([[1.0, 0.1], [0.01, 0.001]], dtype=torch.float)
z.backward(v)
print(x.grad)

tensor([2.0000, 0.2000, 0.0200, 0.0020])


## 中断梯度的例子

In [35]:
x = torch.tensor(0.1, requires_grad=True)
y1 = x ** 2
with torch.no_grad(): # 如下代码关闭自动求导机制
    y2 = x ** 3
y3 = y1 + y2

print(x.requires_grad)
print(y1, y1.requires_grad)
print(y2, y2.requires_grad)
print(y3, y3.requires_grad)

True
tensor(0.0100, grad_fn=<PowBackward0>) True
tensor(0.0010) False
tensor(0.0110, grad_fn=<AddBackward0>) True


In [36]:
y3.backward()
print(x.grad) # y3 = y1 + y2 = x ** 2 + x *** 3, 但是y2被屏蔽求导了，所以最后只有y1

tensor(0.2000)


## 如果我们想要修改 tensor 的数值,但是又不希望被 autograd 记录(即不会影响反向传播),  那么我们可以对 tensor.data 进行操作。

In [43]:
x = torch.ones(1, requires_grad=True)
print(x.data)
print(x.data.requires_grad)

y = 2 * x
x.data *= 100 # 只是改变了值，但是不会影响梯度传播

y.backward()
print(x) # 更改data值也会影响tensor值
print(x.grad)

tensor([1.])
False
tensor([100.], requires_grad=True)
tensor([2.])
